## Exercise 4 - Answer
Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import urllib
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

DESIRED_ACCURACY = 0.999

In [2]:
# colab 환경
# !wget --no-check-certificate \
#     "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
#     -O "/tmp/happy-or-sad.zip"

# zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
# zip_ref.extractall("/tmp/h-or-s")
# zip_ref.close()

In [3]:
# happy-or-sad 데이터셋 다운로드 , Windows용
_TRAIN_URL = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip"
urllib.request.urlretrieve(_TRAIN_URL, 'happy-or-sad.zip')

# 압축해제
local_zip = 'happy-or-sad.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/h-or-s')
zip_ref.close()

In [4]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > DESIRED_ACCURACY):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(rescale=1/255)
# train_datagen = ImageDataGenerator(rescale=1./255.,
#                                   rotation_range=40,
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        "tmp/h-or-s",  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [7]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 4s 262ms/step - loss: 2.3388 - accuracy: 0.6250
Epoch 2/15
8/8 [==============================] - 2s 250ms/step - loss: 0.5485 - accuracy: 0.7875
Epoch 3/15
8/8 [==============================] - 2s 240ms/step - loss: 0.3102 - accuracy: 0.8625
Epoch 4/15
8/8 [==============================] - 2s 249ms/step - loss: 0.2568 - accuracy: 0.8875
Epoch 5/15
8/8 [==============================] - 2s 209ms/step - loss: 0.1447 - accuracy: 0.9500
Epoch 6/15
8/8 [==============================] - 2s 206ms/step - loss: 0.1350 - accuracy: 0.9625
Epoch 7/15
8/8 [==============================] - 2s 230ms/step - loss: 0.1046 - accuracy: 0.9500
Epoch 8/15
8/8 [==============================] - ETA: 0s - loss: 0.0390 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 1s 182ms/step - loss: 0.0390 - accuracy: 1.0000
